## Run gather at k31
- Against merged dbs and normal dbs for all the ones we did before


In [ ]:
# index merged to rocksdb
sourmash index MAGs.all_k31.merged.rocksdb -F rocksdb -k 31 MAGs.all_k31.merged.zip

# make merged db of the gtdb one 
srun --account=ctbrowngrp -p med2 -J mergdb -t 2:00:00 -c 2 --mem=50gb --pty bash

mamba activate pangenomics_dev
sourmash scripts pangenome_merge \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.k31-sc10k.sig.rocksdb \
 -k 31 --scaled 10000 -o gtdb-rs226.merged_k31.rocksdb

sourmash index gtdb-rs226.merged_k31.rocksdb -F rocksdb -k 31 gtdb-rs226.merged_k31.sig




In [ ]:
# normal gather bc 2 dbs
srun --account=ctbrowngrp -p med2 -J calc-pres -t 6:00:00 -c 60 --mem=80gb --pty bash
# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 60 --rerun-incomplete -k -s gather_human.smk -n

In [ ]:
# do fastmultigather with these
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
gtdb-rs226.merged_k31.10k.zip ../sketches/MAGs.all_k31.merged.rocksdb \
-k 31 -s 10000 -c 100 -o gather_k31/250811_fmg_human_gtdbmags.csv

sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
gtdb-rs226.merged_k31.rocksdb \
-k 31 -s 10000 -c 100 -o gather_k31/250811_fmg_human_gtdb.csv

In [ ]:

srun --account=ctbrowngrp -p med2 -J fmg -t 1:00:00 -c 100 --mem=100gb --pty bash

# do fmg against a concatenated database
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
../../sketches/MAGs.all_k31.merged.zip -k 31 -s 10000 -c 100 -t 0 -o ../250808_fmg_MAGs_merged.k31.human.csv



In [ ]:
# we need to add tax to the pig ones. 
# after gather ran tables plugin for presence absense
# for getting f_unique per species (with diff db)
python scripts/250813_specieslvl_funique.py

# need to concat all tax output
csvtk concat *.csv > ../gtdbmags_pig.tax.summarized.csv

# then use python script to get sums at each tax level. 
